In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
sec = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=sec

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
api_key = os.environ.get("PINECONE_API_KEY")
environment = os.environ.get("PINECONE_ENVIRONMENT")
pc = Pinecone(api_key=api_key)

# Ensure the index exists
index_name = "chatinit"  # Replace with your actual index name
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
index = pc.Index(index_name)
vectore_Store = PineconeVectorStore(index=index, embedding=embedding_model)

In [7]:
from langchain.prompts import PromptTemplate

In [8]:
template = """
You are a bot with the following enabled services: {enabled_services}.
Your task is to analyze the user's query and determine whether it falls under the category of "sales," "support," or is a "general" query.
Based on the category, respond accordingly, considering the context of the ongoing conversation.

**Chat History:**
{chat_history}

**User Query:**
{query}

**Instructions:**
1. **Categorization:**
   - Analyze the query and classify it as "sales," "support," or "general."

2. **Response Based on Category:**
   - If the query relates to sales, provide a sales-specific answer with relevant suggestions or information.
   - If the query relates to support, offer guidance or troubleshooting steps tailored to the user's issue.
   - If the query is general or does not relate to the enabled services, provide a general response with helpful insights.

3. **Clarifications:**
   - If the question is unclear or lacks context, politely ask the user for more information to clarify their intent.
   - For example:
     - "Could you provide more details about the issue you're facing?"
     - "Can you clarify what you mean by [specific term or phrase]?"
     - "Are you referring to [specific product/service]?"

4. **Follow-Up Questions:**
   - If the bot cannot answer the query immediately due to insufficient details, ask specific follow-up questions to gather the necessary information.
   - Ensure the questions are simple and focused on resolving the ambiguity.

5. **Tone and Style:**
   - Use a professional and friendly tone.
   - Keep responses concise, clear, and contextually relevant.

6. **Final Note:**
   - If the bot cannot assist even after clarifications, acknowledge the limitation and guide the user to alternative solutions, such as contacting support or visiting a FAQ page.
   - For example:
     - "I might need more details to help you better. Could you clarify this further?"
     - "I'm not sure I can assist with that directly, but you can try reaching out to our support team for additional help."

"""
prompt = PromptTemplate(template=template, input_variables=["enabled_services", "query"])

In [9]:
services_enabled = ["sales", "support"]  # Dynamically provided by the user
"""
Can you help me with a product recommendation of Samsung?
How do I reset my account password?
Who is the founder of Google?
"""

# Format services as a string
services_str = ", ".join(services_enabled)

In [10]:
chain = prompt | llm

In [11]:
user_query = "Who is the founder of Google?"
result = chain.invoke({"enabled_services": services_str, "query": user_query})
chat_history = []

KeyError: "Input to PromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'enabled_services', 'query'] Received: ['enabled_services', 'query']\nNote: if you intended {chat_history} to be part of the string and not a variable, please escape it with double curly braces like: '{{chat_history}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [36]:

while True:
    # Get user input
    user_query = input("\nYou: ")
    if user_query.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    # Append the user's query to the chat history
    chat_history.append(f"User: {user_query}")

    # Format chat history as a string
    chat_history_str = "\n".join(chat_history)

    # Generate response using the chain
    try:
        response = chain.invoke({
            "enabled_services": services_str,
            "chat_history": chat_history_str,
            "query": user_query
        })

        # Append the bot's response to the chat history
        chat_history.append(f"Bot: {response}")
        result = response.content
        # Print the bot's response
        print(f"Chatbot: {result}")

    except Exception as e:
        print(f"Chatbot: Sorry, I encountered an error: {str(e)}")


You: Who is the chief advisor of Bangladesh?
Chatbot: content='This is a general query. The position of "Chief Advisor" in Bangladesh usually refers to the head of a caretaker government, a temporary government installed to oversee elections.  The most recent caretaker government was in power from 2006-2008.  Currently, Bangladesh has a parliamentary system with a Prime Minister as the head of government.  For the most up-to-date information, I recommend checking a reliable news source or government website.\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-46666666-6666-6666-6666-666666666666-0' usage_metadata={'input_tokens': 3032, 'output_tokens': 96, 'total_tokens': 3128, 'input_token_details': {'cache_read': 0}}


You: exit
Chatbot: Goodbye!
